In [88]:
import asyncio

import pandas as pd
from datetime import datetime, timedelta
import pandas
from database import postgres, clickhouse
from schemes import Coin

await postgres.connect()
await clickhouse.connect()

In [2]:
datetime(2025, 2, 23, 21, 49, 13) - timedelta(days=1)

datetime.datetime(2025, 2, 22, 21, 49, 13)

In [79]:
df = pandas.read_excel('Parser_crypto.xlsx', 'spot')
pairs = pd.unique(df['pair'])

In [80]:
ts = (await clickhouse.execute(f"select ts from coins where type='spot' order by ts limit 1")).first_row[0]
ts = (await clickhouse.execute(f"select cast ('2025-02-22 22:57:32' as datetime)")).first_row[0]
ts

datetime.datetime(2025, 2, 22, 22, 57, 32)

In [93]:
async def get_data(type: str) -> tuple[dict, dict]:
    pg_coins = []
    columns = (
        'symbol', 'market_name', 'type', 'price', 'spread', 'index_price', 'volume_24h', 'open_interest',
        'funding_rate',
        'ts')
    for row in (await postgres.fetch(f"""
        SELECT m.name AS market_name, symbol, type, price, spread, index_price, volume_24h,  open_interest, funding_rate, ts
        FROM coins c
        RIGHT JOIN markets m ON m.id = c.market_id
        where type='{type}'
    """)):
        pg_coins.append(Coin(
            market_name=row[0],
            symbol=row[1],
            type=row[2],
            price=row[3],
            spread=row[4],
            index_price=row[5],
            volume_24h=row[6],
            open_interest=row[7],
            funding_rate=row[8],
            ts=row[9]
        ))
    ch_coins = []
    for row in (await clickhouse.execute(f"select * from coins where ts = '{ts}' and type = '{type}'")).result_set:
        ch_coins.append(
            Coin(
                market_name=row[1],
                symbol=row[0],
                type=row[2],
                price=row[3],
                spread=row[4],
                index_price=row[5],
                volume_24h=row[6],
                open_interest=row[7],
                funding_rate=row[8],
                ts=row[9]
            )
        )
    return pg_coins, ch_coins

In [82]:
def get_perc_price(pg: dict, ch: dict):
    return round((pg['price'] - ch['price']) / ch['price'] * 100, 7)


def get_volume_perc(pg: dict, ch: dict) -> float:
    return round((pg['volume_24h'] - ch['volume_24h']) / ch['volume_24h'] * 100, 7)


async def get_df(type: str) -> pd.DataFrame:
    pg_coins, ch_coins = await get_data(type)
    ch_dict = {}
    for coin in ch_coins:
        dct = coin.model_dump()
        dct.pop('symbol')
        ch_dict[coin.symbol] = dct
    pg_dict = {}
    for coin in pg_coins:
        dct = coin.model_dump()
        dct.pop('symbol')
        pg_dict[coin.symbol] = dct
    rows = []
    for pair in pairs:
        pg_data = pg_dict.get(pair)
        if not pg_data:
            continue
        rows.append(
            pd.Series(
                {
                    "market_name": "bybit",
                    "type": "spot",
                    "symbol": pair,
                    "price": pg_data["price"],
                    "price_perc": get_perc_price(pg_data, ch_dict[pair]),
                    "spread": pg_data["spread"],
                    "volume_24h": pg_data["volume_24h"],
                    "volume_24h_perc": get_volume_perc(pg_data, ch_dict[pair]),
                    "open_interest": pg_data["open_interest"],
                    "funding_rate": pg_data["funding_rate"],
                    "ts": pg_data["ts"],
                }
            )
        )
    return pd.DataFrame(rows)

In [96]:
df_spot, df_linear = await asyncio.gather(*[get_df(type) for type in ('spot', 'linear')])

In [97]:
df_linear

,market_name,type,symbol,price,price_perc,spread,volume_24h,volume_24h_perc,open_interest,funding_rate,ts
0,bybit,spot,UXLINKUSDT,0.671400,0.343745,0.014892,1.938462e+07,213.602510,4926174.62,0.000050,2025-02-24 09:25:44.775
1,bybit,spot,BNTUSDT,0.502550,-5.286468,0.039734,1.215894e+06,91.957904,608722.73,0.000100,2025-02-24 09:25:44.775
2,bybit,spot,FBUSDT,1.178000,0.856164,0.084890,8.173564e+05,89.602308,380521.09,0.001430,2025-02-24 09:25:44.775
3,bybit,spot,SPXUSDT,0.702100,-5.249663,0.014259,9.005782e+06,3.223432,4459773.64,0.000050,2025-02-24 09:25:44.775
4,bybit,spot,BOMEUSDT,0.001958,-11.282284,0.102093,2.380363e+09,-21.235357,6878067.22,0.000050,2025-02-24 09:25:44.775
...,...,...,...,...,...,...,...,...,...,...,...
190,bybit,spot,ANKRUSDT,0.022480,-5.980761,0.044464,3.377426e+07,-21.175181,3008260.34,0.000100,2025-02-24 09:25:44.775
191,bybit,spot,MVLUSDT,0.003791,-3.830543,0.184259,7.393170e+07,-78.110992,288549.69,-0.004810,2025-02-24 09:25:44.775
192,bybit,spot,MONUSDT,0.035660,-10.085729,0.028011,2.383034e+07,-12.087581,190445.01,-0.000047,2025-02-24 09:25:44.775
193,bybit,spot,TUSDT,0.020840,-4.883615,0.095923,1.602199e+07,34.295047,562944.88,0.000100,2025-02-24 09:25:44.775


In [99]:
df = pd.read_excel("Parser_crypto.xlsx")
with pd.ExcelWriter("Parser_crypto.xlsx", "openpyxl") as fp:
    df.to_excel(fp, sheet_name="default", index=False)
    df_spot.to_excel(fp, sheet_name="spot", index=False)
    df_linear.to_excel(fp, sheet_name="linear", index=False)